## Dynamic Circuits with OpenQASM 3.0 with Amazon Braket

In this notebook, we show how to utilize dynamic circuit tools for IQM Garnet with Braket using OpenQASM 3.0. 

OpenQASM is an intermediate representation for quantum instructions, used extensively in quantum programming. [One goal](https://arxiv.org/abs/2104.14722) of OpenQASM 3.0 specifically was to support emerging usecases requiring "real-time" classical feedback, which encompasses dynamic circuit constructions. 

### Notebook Setup

Here, we will only use the device configuration in `iqm_config`, as the simulator instructions produce different OpenQASM files. We utilize a `run_qpu` flag for device execution. Notice the additional IR and serialization imports from Braket.  

In [ ]:
from math import pi

import iqm_config

from braket.circuits import Circuit
from braket.circuits.serialization import (
    IRType,
    OpenQASMSerializationProperties,
    QubitReferenceType,
)
from braket.experimental_capabilities import EnableExperimentalCapability
from braket.ir.openqasm import Program
from braket.tracking import Tracker

run_qpu = True

qd = iqm_config.qd
kw = {"tags":iqm_config.tags}

track = Tracker().start()

The following block shows the basic syntax in OpenQASM 3.0 for the `measure_ff` and `cc_prx` instructions included in the `braket.experimental_capabilities`. 
~~~
// mcm_ff.qasm
// perform mid circuit measurement and active qubit reset on a single qubit
OPENQASM 3;
bit[1] c;
#pragma braket verbatim
box{
    prx(3.14, 0) $1;
    measure_ff(0) $1;
    cc_prx(3.14, 0, 0) $1;
    }
c[0] = measure $1;
~~~
Below, we load the local qasm file, view it, and then submit it on the device. 

In [7]:
with open("mcm_ff.qasm", "r") as fp:
    qasm = fp.read()

with EnableExperimentalCapability():
    qasm_prog = Program(source=qasm)
    print(qasm)

    if run_qpu:
        result = qd.run(qasm_prog, shots=100, **kw).result()
        print(f"Measurement counts: {result.measurement_counts}")

// mcm_ff.qasm
// perform mid circuit measurement and active qubit reset 
OPENQASM 3;
bit[1] c;
#pragma braket verbatim
box{
    prx(3.14, 0) $1;
    measure_ff(0) $1;
    cc_prx(3.14, 0, 0) $1;
    }
c[0] = measure $1;

Measurement counts: Counter({'0': 95, '1': 5})


Another way of generating the QASM file is directly from a Braket `Circuit`.

In [5]:
with EnableExperimentalCapability():
    circ = Circuit()
    circ.prx(0,pi,0)
    circ.measure_ff(0,0)
    circ.cc_prx(0, pi, 0, 0)

print(circ)

T  : │       0        │    1    │         2          │
      ┌──────────────┐ ┌───────┐ ┌──────────────────┐ 
q0 : ─┤ PRx(3.14, 0) ├─┤ MFF→0 ├─┤ 0→CCPRx(3.14, 0) ├─
      └──────────────┘ └───────┘ └──────────────────┘ 
T  : │       0        │    1    │         2          │


In [6]:
circ = Circuit().add_verbatim_box(circ)
qasm = circ.to_ir(IRType.OPENQASM, OpenQASMSerializationProperties(QubitReferenceType.PHYSICAL), {}).source
print(qasm)

OPENQASM 3.0;
bit[1] b;
#pragma braket verbatim
box{
prx(3.141592653589793, 0.0) $0;
measure_ff(0) $0;
cc_prx(3.141592653589793, 0.0, 0) $0;
}
b[0] = measure $0;


#### Cost Tracking

The total cost of running these examples is given below:

In [4]:
print("Quantum Task Summary")
print(track.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).",
)
print(
    f"Estimated cost to run this example: {track.qpu_tasks_cost() + track.simulator_tasks_cost():.3f} USD",
)

Quantum Task Summary
{}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 0.000 USD


### References

- OpenQASM live specification. (June 2025). https://openqasm.com/ 
- Cross, Javadi-Abhari, Alexander et al. *OpenQASM 3: A broader and adeeper quantum assembly language*. (2022) [arXiv:2014.14722](https://arxiv.org/abs/2104.14722)
- Run your Circuits with OpenQASM 3.0. Amazon Braket Devleoper Guide (June 2025). https://docs.aws.amazon.com/braket/latest/developerguide/braket-openqasm.html. 